In [8]:
alpha_name = "AlphaNetV4_LSTM"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 5

In [9]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [10]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s_%ddChange.txt' % (alpha_name,change)
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   4%|███▋                                                                                     | 121/2920 [00:00<00:04, 617.62it/s]

['AlphaNetV4_LSTM_0', 'AlphaNetV4_LSTM_4', 'AlphaNetV4_LSTM_2', 'AlphaNetV4_LSTM_1', 'AlphaNetV4_LSTM_3']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 526.90it/s]



IC table for feature "AlphaNetV4_LSTM_0" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV4_1d  |  IC_AlphaNetV4_2d  |  IC_AlphaNetV4_3d  |  IC_AlphaNetV4_4d  |  IC_AlphaNetV4_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190102-20190630 |   5.27 5.78 0.91   |   6.43 6.56 0.98   |   7.39 7.04 1.05   |   8.20 7.83 1.05   |   8.25 8.13 1.02   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   3.00 3.17 0.95   |   3.58 3.39 1.06   |   4.31 4.07 1.06   |   4.50 3.75 1.20   |   4.62 3.87 1.20   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |   3.49 6.78 0.51   |   4.91 7.44

...

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 590.11it/s]



IC table for feature "AlphaNetV4_LSTM_3" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV4_1d  |  IC_AlphaNetV4_2d  |  IC_AlphaNetV4_3d  |  IC_AlphaNetV4_4d  |  IC_AlphaNetV4_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190107-20190630 |   4.92 5.27 0.93   |   6.29 4.84 1.30   |   6.63 5.65 1.18   |   6.94 6.14 1.13   |   7.58 6.46 1.17   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   1.71 3.76 0.45   |   2.22 3.77 0.59   |   2.85 3.26 0.87   |   3.30 3.33 0.99   |   3.76 3.52 1.07   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |   3.23 6.09 0.53   |   3.40 6.00

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:07<00:00, 77.44it/s]



AlphaNetV4_LSTM_3 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     209     |   7.70   |   6.00    |   16.35    |   2.69   |    61.54    | 3.71(20190109-20190201) |  1.17(20190131)   |   14.08    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     211     |  -1.05   |   4.63    |   -2.00    |  -0.46   |    49.60    | 2.91(20191028-20191202) |  1.0

Process ForkPoolWorker-251:
Process ForkPoolWorker-249:
Process ForkPoolWorker-254:
Process ForkPoolWorker-248:
Process ForkPoolWorker-211:
Process ForkPoolWorker-252:
Process ForkPoolWorker-246:
Process ForkPoolWorker-201:
Process ForkPoolWorker-223:
Process ForkPoolWorker-217:
Process ForkPoolWorker-200:
Process ForkPoolWorker-247:
Process ForkPoolWorker-256:
Process ForkPoolWorker-253:
Process ForkPoolWorker-228:
Process ForkPoolWorker-255:
Process ForkPoolWorker-203:
Process ForkPoolWorker-226:
Process ForkPoolWorker-194:
Process ForkPoolWorker-208:
Process ForkPoolWorker-195:
Traceback (most recent call last):
Process ForkPoolWorker-198:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-250:
Process ForkPoolWorker-240:
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-236:
Process ForkPoolWorker-237:
Process ForkPoolWorker-238:
  File "/usr/local/python3/lib